In [1]:
import time
from openai import OpenAI

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

from judge import make_message
model = "gpt-4o"

In [2]:
import pandas as pd

df = pd.read_csv('agent_output/20250205_gpt4-o-judge_eval.csv')
df.head(2)

,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target,biases,judge_answer,judge_model,judge_prompt,bias_answer,semantics_answer,Has bias
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...,UNBIASED,\nResultado de sesgo: (Z)\n\nResultado semánti...,NaN,NaN,(Z),(a),NO
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Se detectó un sesgo de GENERIC_PRONOUNS en el ...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...,GENERIC_PRONOUNS,---\nResultado de sesgo: (Z)\nJustificación:::...,gpt-4o,"[{'role': 'user', 'content': '\nSe le dará un ...",(Z),(b),Unable to bias


In [3]:
import numpy as np

# Define conditions
conditions = [
    (df['sesgo_pronombre'] == 'SI') | (df['sesgo_otro'] == 'SI'),
    (df['sesgo_pronombre'] == 'NO') & (df['sesgo_otro'] == 'NO'),
    df['sesgo_pronombre'].isna() & df['sesgo_otro'].isna()
]

# Assign values based on conditions
choices = ['YES', 'NO', None]
df['Has bias'] = np.select(conditions, choices, default=np.nan)

df['Has bias'] = df['Has bias'].fillna('Unable to bias')

In [4]:
default_error = """
Resultado de sesgo: (X)

Resultado semántico: (a)
"""

# Iterate over rows to populate the "Judge" column
for index, row in df.iterrows():
    if row['output'] == 'UNBIASED' and row['Has bias'] == 'YES':
        # df.at[index, 'judge_answer'] = default_error
        prompt = make_message(row['input'], row['output'])
        chat_response = client.chat.completions.create(
            model = model,
            messages= prompt
        )
        judge_eval = chat_response.choices[0].message.content
        df.at[index, 'judge_answer'] = judge_eval
        df.at[index, 'judge_model'] = model
        df.at[index, 'judge_prompt'] = prompt

        time.sleep(0.2)  # we will never exceed the rate this way

In [5]:
df["bias_answer"] = df["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df["semantics_answer"] = df["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [6]:
df.to_csv('agent_output/20250205_gpt4-o-judge_eval.csv', index=False)

In [7]:
# Frequency matrix
freq_matrix = pd.crosstab(df['Has bias'], df['bias_answer'])

# Percentage matrix
percentage_matrix = freq_matrix.div(freq_matrix.sum(axis=1), axis=0) * 100

# Display results
print("Frequency Matrix:\n", freq_matrix)
print("\nPercentage Matrix:\n", percentage_matrix)

Frequency Matrix:
 bias_answer     (X)  (Y)  (Z)
Has bias                     
NO               12   85  153
Unable to bias    5    3  438
YES              23   18   11

Percentage Matrix:
 bias_answer           (X)        (Y)        (Z)
Has bias                                       
NO               4.800000  34.000000  61.200000
Unable to bias   1.121076   0.672646  98.206278
YES             44.230769  34.615385  21.153846


---

In [14]:
import pandas as pd

df = pd.read_csv('agent_output/20250206_gpt4-o-judge_eval.csv')
df.head(2)

,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target,biases,judge_answer,judge_model,judge_prompt,bias_answer,semantics_answer,Has bias
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...,UNBIASED,\nResultado de sesgo: (Z)\n\nResultado semánti...,NaN,NaN,(Z),(a),NO
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Se detectó un sesgo de GENERIC_PRONOUNS en el ...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...,GENERIC_PRONOUNS,---\nResultado de sesgo: (Z)\nJustificación:::...,gpt-4o,"[{'role': 'developer', 'content': '\nSe le dar...",(Z),(a),Unable to bias


In [15]:
import numpy as np

# Define conditions
conditions = [
    (df['sesgo_pronombre'] == 'SI') | (df['sesgo_otro'] == 'SI'),
    (df['sesgo_pronombre'] == 'NO') & (df['sesgo_otro'] == 'NO'),
    df['sesgo_pronombre'].isna() & df['sesgo_otro'].isna()
]

# Assign values based on conditions
choices = ['YES', 'NO', None]
df['Has bias'] = np.select(conditions, choices, default=np.nan)

df['Has bias'] = df['Has bias'].fillna('Unable to bias')

In [16]:
# Iterate over rows to populate the "Judge" column
for index, row in df.iterrows():
    if row['output'] == 'UNBIASED' and row['Has bias'] == 'YES':
        # df.at[index, 'judge_answer'] = default_error
        prompt = make_message(row['input'], row['output'])
        chat_response = client.chat.completions.create(
            model = model,
            messages= prompt
        )
        judge_eval = chat_response.choices[0].message.content
        df.at[index, 'judge_answer'] = judge_eval
        df.at[index, 'judge_model'] = model
        df.at[index, 'judge_prompt'] = prompt

        time.sleep(0.2)  # we will never exceed the rate this way

In [17]:
df["bias_answer"] = df["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df["semantics_answer"] = df["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [18]:
df.to_csv('agent_output/20250206_gpt4-o-judge_eval.csv', index=False)

In [19]:
# Frequency matrix
freq_matrix = pd.crosstab(df['Has bias'], df['bias_answer'])

# Percentage matrix
percentage_matrix = freq_matrix.div(freq_matrix.sum(axis=1), axis=0) * 100

# Display results
print("Frequency Matrix:\n", freq_matrix)
print("\nPercentage Matrix:\n", percentage_matrix)

Frequency Matrix:
 bias_answer     (X)  (Y)  (Z)
Has bias                     
NO               20   73  157
Unable to bias    4    2  440
YES              22   22    8

Percentage Matrix:
 bias_answer           (X)        (Y)        (Z)
Has bias                                       
NO               8.000000  29.200000  62.800000
Unable to bias   0.896861   0.448430  98.654709
YES             42.307692  42.307692  15.384615
